In [ ]:
# 포켓몬 이미지 분류기 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')                                       

# Data loading 

In [ ]:
# 판다스와 os 라이브러리를 불러온다
import pandas as pd
import os 
base_dir = '/content/drive/My Drive/Colab Notebooks' # 기본 디렉터리를 base_dir 변수에 저장

pokemons = pd.read_csv(base_dir+'/pokemon.csv') # pokemons 변수에 csv를 넣는다

numbers = [] # 빈 리스트를 만듦
for i in range(1,pokemons.shape[0]+1): # 1부터 csv 파일의 행 수 + 1까지 i 를 증가시켜 가며 반복
    numbers.append(i) # numbers에 새롭게 붙인다
pokemons['pkn'] = numbers # pokemon 데이터프레임에 pkn이라는 변수에 numbers 리스트를 저장한다.
IMG_DIR = base_dir + '/images/images' # IMG_DIR 변수에 '기본 디렉터리경로 + 이미지 폴더 내 이미지 폴더 경로'를 삽입


from os import listdir #os 라이브러리 내 listdir 메소드를 가져온다.

from os.path import isfile, join #os.path 라이브러리 내에 isfile, join 메소드를 가져온다

onlyfiles = [f for f in listdir(IMG_DIR) if isfile(join(IMG_DIR, f))] #onlyfiles 변수에 ?????????

In [ ]:
join(IMG_DIR, 'tyrunt.png')

In [ ]:
listdir(IMG_DIR)

In [ ]:
pokemons

In [ ]:
onlyfiles

In [ ]:
pkn

In [ ]:
import re
dataframe_img = pd.DataFrame([]) # 빈 데이터프레임을 하나 만들고 dataframe_img라는 이름의 변수 이름을 지정
images = [] # images라는 빈 리스트를 만듦
pokemon_name = [] # pokemon_name 이라는 빈 리스트를 만듦
for img in onlyfiles: # onlyfiles 내에 있는 img 각각에 대해 아래 사항을 반복
    if not re.search('-', img): # onlyfiles 안에 있는 img에서 '-'를 찾아서 없으면???
        pkn = img.split('.') # pkn에 img를 .으로 나누어서 넣어라 ex) swalot.png 에서 .을 기준으로 앞에꺼는 name에 넣고, 
        name =  pkn[0]
        path = IMG_DIR +'/' +str(img) #이미지 경로는 '기본 디렉터리경로 + 이미지 폴더 내 이미지 폴더 경로'/swalot.png
        images.append(path) # images라는 리스트에 path를 하나씩 붙인다
        pokemon_name.append(name) #pokemon_name이라는 리스트에 name을 하나씩 붙인다
dataframe_img['images'] = images #dataframe_img라는 빈 데이터프레임 내에 images라는 변수에 images 리스트(path로 채웠음)를 넣는다
dataframe_img['Name'] = pokemon_name #dataframe_img라는 빈 데이터프레임 내에 'Name'이라는 변수에 pokemon_name 리스트(포켓몬 이름으로 채웠음)를 넣는다.

result = pokemons.merge(dataframe_img, left_on='Name', right_on='Name') # result 라는 변수에 pokemons 데이터프레임과 방금 만든 dataframe_img를 좌우병합. 병합 시 기준은 모두 Name 변수
result.head() # result의 첫 5개 행을 보자.

############################################################호섭님
img = []

for pokemon in pokemons['Name']:
    if str(pokemon)+'.png' in onlyfiles:
        img.append('data/images/images/'+str(pokemon)+'.png')
    else:
        img.append('data/images/imgeas/'+str(pokemon)+'.jpg')
        
pokemons['img'] = img


# 시각화

In [ ]:
import seaborn as sns #seaborn 라이브러리를 불러온다
import matplotlib.pyplot as plt # matplotlib.pyplot을 불러온다

select = ['Water', 'Fire'] # 물과 불로 이루어진 리스트를 select 변수에 넣음
result = result[result['Type1'].isin(select)] 
# result의 Type1 변수에 Water나 Fire가 들어가있는 데이터를 result로 만듦. 덮어씀. 왜?
sns.countplot(x='Type1', data=result) # 빈도 그래프. x축 이름, 데이터 지정
plt.title("Water vs Fire, count plot") # 그래프 제목 달기
plt.show() # 그래프를 화면에 표시

# split data 

In [ ]:
import os
from shutil import copyfile
os.mkdir('train/')
os.mkdir('test/')
os.mkdir('val/')
for class_ in result['Type1'].unique():
    os.mkdir('train/'+str(class_)+'/')
    os.mkdir('test/'+str(class_)+'/')
    os.mkdir('val/'+str(class_)+'/')

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    result, result['Type1'],test_size=0.33, stratify=result['Type1'])

X_test, X_val, y_test, y_val = train_test_split(
    X_test, y_test, test_size=0.33,stratify=y_test)

In [ ]:
from shutil import copyfile, copy2

for image,type_  in zip(X_train['images'], y_train):
    copy2(image, 'train/'+type_)

for image,type_ in zip(X_test['images'], y_test):
    copy2(image, 'test/'+type_)
    
for image,type_ in zip(X_val['images'], y_val):
    copy2(image, 'val/'+type_)

In [ ]:
# ModuleNotFoundError: No module named 'keras' 해결
pip install  keras

In [ ]:
# No module named 'tensorflow' 해결
#pip install tensorflow --ignore-installed --user
#pip install tensorflow --user
#pip show tensorflow
conda create -n tensorflow

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator()

train = datagen.flow_from_directory('train/')
test = datagen.flow_from_directory('test/')
val = datagen.flow_from_directory('val/')

# modeling

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization, Lambda
from keras.preprocessing.image import ImageDataGenerator

def build():
    model = Sequential()
    IMAGE_WIDTH = 256
    IMAGE_HEIGHT = 256
    IMAGE_CHANNELS = 3
    model.add(Lambda(lambda x: x, input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))
    model.add(Conv2D(32, (2, 2), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (2, 2), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (2, 2), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax')) 

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

    model.summary()
    return model
model = build()
history = model.fit_generator(train, epochs=30, validation_data=val)

In [ ]:
predict = model.predict_generator(test)

In [ ]:
from sklearn.metrics import classification_report
import numpy as np 
predict_frame = pd.DataFrame([])
predict_frame['category'] = np.argmax(predict, axis=-1)
labels = dict((v,k) for k,v in val.class_indices.items())
predict_frame['category'] = predict_frame['category'].replace(labels)
print(classification_report(y_test, predict_frame['category']))

# test

In [ ]:
def show_wrong_classification(y_test, predict, result):
    tmp = result[result.index.isin(y_test.index)]
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(20, 20))
    i=0
    for imag, true, pred in zip(tmp['images'], tmp['Type1'], predict):
        if true!=pred:
            if i <3:
                img = Image.open(imag)
                fig = plt.figure()
                ax[i].imshow(img)
                ax[i].set_title(str(pred))
                i+=1

In [ ]:
from PIL import Image
show_wrong_classification(y_test, predict_frame['category'], result)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2,3,figsize=(15,10))
k =0
import cv2
list_b =[]
list_r = []
list_g = []
from tqdm import tqdm
for type_ in tqdm(result['Type1'].unique()):
    tmp = result[result['Type1']==type_]
    for img in tmp['images']:
        img = cv2.imread(img)
        b, g, r = cv2.split(img)
        color = 'blue'
        for i in b:
            for j in i:
                if j != 0:
                    list_b.append(j)
        color = 'green'
        for i in g:
            for j in i:
                if j != 0:
                    list_g.append(j)
        color = 'red'
        for i in r:
            for j in i:
                if j != 0:
                    list_r.append(j)
    sns.distplot(list_g, ax=axes[k, 0], color='g')
    sns.distplot(list_b, ax=axes[k, 1], color='b')
    sns.distplot(list_r, ax=axes[k, 2], color='r')
    axes[k, 0].set_title('Pokemon type color channel ' + type_)
    if type_ =='Fire':
        list_g_f = list_g
        list_b_f = list_b
        list_r_f = list_r
    else:
        list_g_w = list_g
        list_b_w = list_b
        list_r_w = list_r
    list_b =[]
    list_r = []
    list_g = []
    
    k += 1

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

g_fire = np.std(np.array(list_g_f), axis=0)
r_fire = np.std(np.array(list_r_f), axis=0)
b_fire = np.std(np.array(list_b_f), axis=0)


g_water = np.std(np.array(list_g_w), axis=0)
r_water = np.std(np.array(list_r_w), axis=0)
b_water = np.std(np.array(list_b_w), axis=0)

fig = make_subplots(rows=1, cols=2, subplot_titles=("Fire std channel","Water std channel"))
fig.add_trace(go.Scatter(
    y=[b_fire, g_fire, g_fire],
    x=['blue', 'green', 'red'],
    mode='markers',
    marker=dict(size=[b_fire, g_fire, r_fire],
                color=['blue', 'green', 'red'],
                showscale=True)
), row=1, col=1)

fig.add_trace(go.Scatter(
    y=[b_water, g_water, r_water],
    x=['blue', 'green', 'red'],
    mode='markers',
    marker=dict(size=[b_water, g_water, r_water],
                color=['blue', 'green', 'red'],
                showscale=True)
), row=1, col=2)
fig.update_layout(showlegend=False)
fig.update_layout({
        'plot_bgcolor': 'rgba(0, 0, 0, 0)',
        'paper_bgcolor': 'rgba(0, 0, 0, 0)',
        'width': 850, 
        'height': 400

})
fig.show()

In [ ]:
water = result[result['Type1']=='Water']
stds = []
values = []
for image in water['images']:
    img = cv2.imread(image)
    b, g, r = cv2.split(img)
    for i in r:
        for j in i:
            if j != 0:
                stds.append(j)       
    std = np.mean(np.array(stds), axis=0)
    values.append(std)
    stds = []
water['stds'] = values

In [ ]:
water = water.sort_values(by='stds', ascending=False)

In [ ]:
i = 0
j = 0
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(20, 20))
for imag in water['images']:
    if i == 1 and j ==3:
        break
    if j > 2:
        i =1
        j = 0
    img = Image.open(imag)
    fig = plt.figure()
    ax[i][j].imshow(img)
    j+=1

In [ ]:
fire = result[result['Type1']=='Fire']
stds = []
values = []
for image in fire['images']:
    img = cv2.imread(image)
    b, g, r = cv2.split(img)
    for i in b:
        for j in i:
            if j != 0:
                stds.append(j)       
    std = np.mean(np.array(stds), axis=0)
    values.append(std)
    stds = []
fire['stds'] = values
fire = fire.sort_values(by='stds', ascending=False)
i = 0
j = 0
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(20, 20))
for imag in fire['images']:
    if i == 1 and j ==3:
        break
    if j > 2:
        i =1
        j = 0
    img = Image.open(imag)
    fig = plt.figure()
    ax[i][j].imshow(img)
    j+=1

## next time

# Data Augmentation 

##brightness

In [ ]:
# example of horizontal shift image augmentation
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot
img = load_img(water['images'][170])
data = img_to_array(img)
samples = expand_dims(data, 0)
datagen = ImageDataGenerator(brightness_range=[0.2,1.5])
it = datagen.flow(samples, batch_size=1)
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(20, 20))
for i in range(3):
    fig = plt.figure()
    batch = it.next()
    image = batch[0].astype('uint8')
    ax[i].imshow(image)
pyplot.show()

## zoom

In [ ]:
img = load_img(fire['images'][156])
data = img_to_array(img)
samples = expand_dims(data, 0)
datagen = ImageDataGenerator(zoom_range=[0.5, 1.0])
it = datagen.flow(samples, batch_size=1)
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(20, 20))
for i in range(3):
    fig = plt.figure()
    batch = it.next()
    image = batch[0].astype('uint8')
    ax[i].imshow(image)
pyplot.show()

## rotation 

In [ ]:
img = load_img(water['images'][118])
data = img_to_array(img)
samples = expand_dims(data, 0)
datagen = ImageDataGenerator(rotation_range=35)
it = datagen.flow(samples, batch_size=1)
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(20, 20))
for i in range(3):
    fig = plt.figure()
    batch = it.next()
    image = batch[0].astype('uint8')
    ax[i].imshow(image)
pyplot.show()

In [ ]:
datagentrain = ImageDataGenerator(rotation_range=35, 
                                 zoom_range=[0.5, 1.0], 
                                 brightness_range=[0.2,1.5])

datagen = ImageDataGenerator()

train = datagentrain.flow_from_directory('train/')
test = datagen.flow_from_directory('test/')
val = datagen.flow_from_directory('val/')

In [ ]:
model = build()
history = model.fit_generator(train, epochs=30, validation_data=val)

In [ ]:
fig = go.Figure()
epochs = []
for i in range(len(history.history['acc'])):
    epochs.append(i)
fig.add_trace(go.Scatter(x=epochs,y=history.history['acc'], mode='lines',name='train'))
fig.add_trace(go.Scatter(x=epochs,y=history.history['val_acc'], mode='lines',name='val'))
fig.update_layout({
        'plot_bgcolor': 'rgba(0, 0, 0, 0)',
        'paper_bgcolor': 'rgba(0, 0, 0, 0)',
        'width': 500, 
        'height': 400

})
fig.show()

In [ ]:
predict_frame = pd.DataFrame([])
predict = model.predict_generator(test)
predict_frame['category'] = np.argmax(predict, axis=-1)
labels = dict((v,k) for k,v in val.class_indices.items())
predict_frame['category'] = predict_frame['category'].replace(labels)
print(classification_report(y_test, predict_frame['category']))

In [ ]:
show_wrong_classification(y_test, predict_frame['category'], result)